# Data Wrangling

*Erin Cameron*

---

## 1.0) Setup

In [ ]:
# Install packages
# !pip install pandas

In [3]:
# Import packages
import pandas as pd

## 2.0) Loading and cleaning data

### 2.1) Dispatch Data

    Dispatch_Time - Time first Paramedic unit assigned to incident (year-month-day hour:minute:seconds)
    ID - Incident ID number associated with Computer Aided Dispatch (CAD) record
    Incident_Type - Type of incident
    Priority_Number - Triaged priority of incident
    Units_Arrived_At_Scene - Number of Paramedic units that arrived on scene of incident
    FSA - General location of incident based on Postal Code Forward Sortation Area

In [21]:
# Read the dispatch dataset in with time-indexing
dispatches = pd.read_excel(r'./data/tps_incident_data_2010-2019.xlsx', parse_dates=True, index_col="Dispatch_Time")
dispatches = dispatches.sort_index()
print(dispatches.shape)

(204808, 5)


In [22]:
display(dispatches)

,ID,Incident_Type,Priority_Number,Units_Arrived_At_Scene,FSA
Dispatch_Time,,,,,
2010-01-01 00:01:36,3061660,Medical,4,2,M5R
2010-01-01 00:04:23,3061663,Medical,1,1,M5V
2010-01-01 00:05:52,3061664,Medical,5,1,M5N
2010-01-01 00:09:53,3061667,Medical,1,1,M4Y
2010-01-01 00:10:36,3061668,Medical,1,2,M6K
...,...,...,...,...,...
2010-12-31 23:57:24,3364485,Medical,1,1,M6K
2010-12-31 23:57:27,3364487,Medical,1,1,M1W
2010-12-31 23:58:03,3364484,Medical,1,1,M5V


In [23]:
# Check the index is sorted before performing index slicing
dispatches.index.is_monotonic_increasing

True

In [ ]:
# Subset for only 2010 Dispatch_Times
dispatches_df = dispatches.loc["2010-01-01 00:01:36":"2010-12-31 23:57:27"].copy()

# Remove any missing data points
dispatches_df.fillna(value=0, inplace=True)
dispatches_df.dropna()

In [32]:
# Subset the data for the most recent year of calls (in 2010)
call_counts = dispatches_df.FSA.value_counts().rename_axis('FSA').reset_index(name='counts')
display(call_counts)

,FSA,counts
0,M6K,4481
1,M5A,4202
2,M4C,3846
3,M1E,3809
4,M1P,3802
...,...,...
150,N4S,1
151,K0K,1
152,L1Z,1
153,L1C,1


In [33]:
# Check if there are any missing values
print(call_counts["FSA"].isnull().sum())

0


### 2.2) Population Data

In [5]:
# Read the Canada population dataset in
population = pd.read_excel(r'./data/canada_population_by_postal.xlsx')
population.sort_index()
population.head()

,Geographic code,Geographic name,Province or territory,"Incompletely enumerated Indian reserves and Indian settlements, 2016","Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016"
0,A0A,A0A,Newfoundland and Labrador,NaN,46587.0,26155.0,19426.0
1,A0B,A0B,Newfoundland and Labrador,NaN,19792.0,13658.0,8792.0
2,A0C,A0C,Newfoundland and Labrador,NaN,12587.0,8010.0,5606.0
3,A0E,A0E,Newfoundland and Labrador,NaN,22294.0,12293.0,9603.0
4,A0G,A0G,Newfoundland and Labrador,NaN,35266.0,21750.0,15200.0


In [35]:
# Rename the population column to the same as the Dispatch column: FSA
rename_pop_column = population.rename(columns = {'Geographic code': 'FSA'}, inplace = False)
display(rename_pop_column.head())

,FSA,Geographic name,Province or territory,"Incompletely enumerated Indian reserves and Indian settlements, 2016","Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016"
0,A0A,A0A,Newfoundland and Labrador,NaN,46587.0,26155.0,19426.0
1,A0B,A0B,Newfoundland and Labrador,NaN,19792.0,13658.0,8792.0
2,A0C,A0C,Newfoundland and Labrador,NaN,12587.0,8010.0,5606.0
3,A0E,A0E,Newfoundland and Labrador,NaN,22294.0,12293.0,9603.0
4,A0G,A0G,Newfoundland and Labrador,NaN,35266.0,21750.0,15200.0


In [36]:
# Locate only the relevant postal code areas
call_counts = pd.merge(call_counts, rename_pop_column, on="FSA")
call_counts = call_counts.rename(columns = {'Population, 2016': 'population'}, inplace = False)
call_counts["call_per_capita"] = call_counts["counts"] / call_counts["population"]
display(call_counts.head())

,FSA,counts,Geographic name,Province or territory,"Incompletely enumerated Indian reserves and Indian settlements, 2016",population,"Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016",call_per_capita
0,M6K,4481,M6K,Ontario,NaN,40957.0,23344.0,22023.0,0.109407
1,M5A,4202,M5A,Ontario,NaN,41078.0,24186.0,22333.0,0.102293
2,M4C,3846,M4C,Ontario,NaN,46866.0,20178.0,19431.0,0.082064
3,M1E,3809,M1E,Ontario,NaN,46943.0,17637.0,17161.0,0.081141
4,M1P,3802,M1P,Ontario,NaN,45571.0,17129.0,16540.0,0.083430


In [37]:
# Identify the outliers
display(call_counts.sort_values("call_per_capita").tail())

,FSA,counts,Geographic name,Province or territory,"Incompletely enumerated Indian reserves and Indian settlements, 2016",population,"Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016",call_per_capita
23,M5B,2901,M5B,Ontario,NaN,12785.0,8249.0,7058.0,0.226907
15,M5G,3143,M5G,Ontario,NaN,8423.0,5876.0,4929.0,0.373145
81,M5H,1070,M5H,Ontario,NaN,2005.0,1718.0,1243.0,0.533666
125,L4V,5,L4V,Ontario,NaN,5.0,1.0,1.0,1.000000
103,L5T,108,L5T,Ontario,NaN,5.0,10.0,4.0,21.600000


In [38]:
# Subset data to remove outlier datapoints at index 125 and 103 - data that is outside of Toronto boundaries
call_counts = call_counts[(call_counts["call_per_capita"] > 0.000000) & (call_counts["call_per_capita"] <= 0.533666)]
display(call_counts.sort_values("call_per_capita").tail())

,FSA,counts,Geographic name,Province or territory,"Incompletely enumerated Indian reserves and Indian settlements, 2016",population,"Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016",call_per_capita
32,M5T,2633,M5T,Ontario,NaN,17748.0,9480.0,8537.0,0.148355
95,M5C,469,M5C,Ontario,NaN,2951.0,1879.0,1721.0,0.158929
23,M5B,2901,M5B,Ontario,NaN,12785.0,8249.0,7058.0,0.226907
15,M5G,3143,M5G,Ontario,NaN,8423.0,5876.0,4929.0,0.373145
81,M5H,1070,M5H,Ontario,NaN,2005.0,1718.0,1243.0,0.533666


### 3.0) File Compression: transform raw data to parquet/csv format

In [39]:
# Output the dispatch data to parquet for compressed size
call_counts.to_parquet('../data/tps_dispatch_counts_2010.parquet', compression=None)

In [46]:
dispatches_df.to_csv('./data/tps_incident_data_2010.csv')